# Hosting models on Grid

Grid offers both: Machine Learning as a Service and Encrypted Machine Learning as a service. In this series of tutorials we show how you can serve and query models on Grid.

## 2.1 Host and query a model using a Rest API

In the previous tutorial we trained a CNN for classifying images with different 2 types of skin deseases: benign keratosis and melanoma (type of skin cancer). In this tutorial we show how to serve this model on Grid using a Rest API.

**Import dependencies**

In [ ]:
import grid as gr
import helper
import syft as sy
import torch

**Setup Config**

Define Config parameters, init hook, etc...

In [ ]:
hook = sy.TorchHook(torch)

### Connect with a remote worker

In [ ]:
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
bob.connect()

### Load dataset

In [15]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (torch.tensor([0.6979, 0.5445, 0.5735]), torch.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:483: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


### Define Model

Let's load the model we just trained.

In [ ]:
model = helper.make_model()
model.load_state_dict(torch.load("binary-skin-cancer-detection-model"))
model.eval()

In [ ]:
helper.test(model, test_generator)

### Making a model ready to be served

In order to serve the model it needs to be serializable. We support two ways of supporting serializitation for models:

1. Using Jit modules. We can turn a regular torch model into a jit module. Jit modules use Torchscript.

> Torchsript creates serializable and optimizable models from PyTorch code. Any code written in TorchScript can be saved from a Python process and loaded in a process where there is no Python dependency. This facility will allow us to send this model to remote workers. - jit documentation

**IMPORTANT**: you'll need torch 1.0.1 in order to run this demo with jit modules due to compatibility issues between syft and torch.jit in the more recent versions.

2. Using Plans. A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. You can learn more about plans in [Syft's tutorials](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2008%20-%20Introduction%20to%20Plans.ipynb).

#### 1. Jit Module

We can turn a regular module into a jit module using torch.jit.trace.

In [ ]:
traced_model = torch.jit.trace(model, data)

In [ ]:
traced_model

#### 2. Plan

We can't turn an existing model into a Plan, in order to create a Plan we need to inherit from syft.Plan during the creation of the model. We implemented this logic at `helper.py`.

In [16]:
plan_model = helper.make_model(is_plan=True)
# plan_model.load_state_dict(torch.load("binary-skin-cancer-detection-model"))
plan_model

<Net Net id:convnet owner:me>

Before sending the plan anywhere we need to build it (similar to the trace operation).

In [ ]:
plan_model.build(data)

## Serve model

We can check the models served at `bob` at `bob.models`.

In [14]:
bob.models

[]

We can serve a new model at `bob` by calling `bob.serve_model(<model>, <an unique model identifier>)`

In [18]:
bob.serve_model(plan_model, model_id="skin-cancer-model-1")

'{"success": true}'

In [19]:
bob.models

['skin-cancer-model-1']

In [ ]:
bob.serve_model(traced_model, model_id="skin-cancer-model-2")

In [ ]:
bob.models

## Query model

Now any one that can connect to this worker can query this model. Let's restart this notebook (so we loose connection to the worker and don't know the model anymore).

In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

First let's reconnect to the worker.

In [12]:
import grid as gr
import torch
import syft as sy
import helper

hook = sy.TorchHook(torch)

bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
bob.connect()

Now we can run inference at the host models by calling `bob.run_inference(model_id, data)`.

In [ ]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (torch.tensor([0.6979, 0.5445, 0.5735]), torch.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

In [20]:
bob.run_inference(model_id="skin-cancer-model-1", data=data), target

({'prediction': [[0.030585136264562607, -0.015594672411680222]]}, tensor([1]))

In [ ]:
bob.run_inference(model_id="skin-cancer-model-2", data=data), target

### Other operations


**Delete a Model**

One can also delete a model if they want.

In [ ]:
# deletes the remote model
bob.delete_model("skin-cancer-model-1")

In [ ]:
bob.models

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

## Star PySyft on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

[Star PySyft](https://github.com/OpenMined/PySyft)

## Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

## Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

[PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
[Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

## Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)